In [2]:
#@title  Package imports

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import pandas as pd

# ML related imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
#@title Importing data from google drive

#from google.colab import drive
#drive.mount('/content/drive')

#points_df = pd.read_csv('/content/drive/MyDrive/MTP Work/AI Dissertation/Python Notebooks/Fractal ML/points_file')

In [3]:
#@title Using uploaded data (live)

df = pd.read_csv('points_file')

In [4]:
# Assuming your DataFrame is named 'df'
data = df.columns.values

# Convert the data to complex floats
data_array = np.array([complex(x) for x in data])

In [5]:
points = data_array

# Assuming 'points' is your array of complex values
real_parts = np.real(points)
imag_parts = np.imag(points)

boundary_points = np.column_stack((real_parts, imag_parts))

In [15]:
# Generate non-boundary points and labels
x_min, x_max = -2, 0.5
y_min, y_max = -1, 1
#resolution = 0.00005  # Adjust this value as needed
resolution = 0.001

boundary_labels = torch.ones(len(boundary_points), dtype=torch.long)
boundary_points = torch.tensor(boundary_points)
print(len(boundary_points))

163045


<ipython-input-15-046cfcff5d71>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  boundary_points = torch.tensor(boundary_points)


In [16]:
x_vals = torch.arange(x_min, x_max, resolution)
y_vals = torch.arange(y_min, y_max, resolution)
xx, yy = torch.meshgrid(x_vals, y_vals)
print('Successfully made meshgrid')
non_boundary_points = torch.stack([xx.flatten(), yy.flatten()], dim=1)
non_boundary_labels = torch.zeros(len(non_boundary_points), dtype=torch.long)
print(len(non_boundary_points))

Successfully made meshgrid
5000000


In [12]:
print(len(boundary_points))
print(len(non_boundary_points))
print(non_boundary_points[51])

163045
50000
tensor([-2.0000, -0.4900])


In [13]:
non_boundary_indices = []

for point in tqdm(non_boundary_points):
  if point in boundary_points:
    continue
  else:
    non_boundary_indices.append(point)

100%|██████████| 50000/50000 [01:54<00:00, 438.39it/s]


[tensor([-2., -1.]), tensor([-2.0000, -0.9900]), tensor([-2.0000, -0.9800]), tensor([-2.0000, -0.9700]), tensor([-2.0000, -0.9600]), tensor([-2.0000, -0.9500]), tensor([-2.0000, -0.9400]), tensor([-2.0000, -0.9300]), tensor([-2.0000, -0.9200]), tensor([-2.0000, -0.9100]), tensor([-2.0000, -0.9000]), tensor([-2.0000, -0.8900]), tensor([-2.0000, -0.8800]), tensor([-2.0000, -0.8700]), tensor([-2.0000, -0.8600]), tensor([-2.0000, -0.8500]), tensor([-2.0000, -0.8400]), tensor([-2.0000, -0.8300]), tensor([-2.0000, -0.8200]), tensor([-2.0000, -0.8100]), tensor([-2.0000, -0.8000]), tensor([-2.0000, -0.7900]), tensor([-2.0000, -0.7800]), tensor([-2.0000, -0.7700]), tensor([-2.0000, -0.7600]), tensor([-2.0000, -0.7500]), tensor([-2.0000, -0.7400]), tensor([-2.0000, -0.7300]), tensor([-2.0000, -0.7200]), tensor([-2.0000, -0.7100]), tensor([-2.0000, -0.7000]), tensor([-2.0000, -0.6900]), tensor([-2.0000, -0.6800]), tensor([-2.0000, -0.6700]), tensor([-2.0000, -0.6600]), tensor([-2.0000, -0.6500]),

In [14]:
print(len(non_boundary_indices))

50000


In [ ]:
non_boundary_points = non_boundary_points[non_boundary_indices]
non_boundary_labels = non_boundary_labels[non_boundary_indices]

In [ ]:
# Remove boundary points from non-boundary points
#non_boundary_mask = ~torch.any(non_boundary_points.unsqueeze(1) == boundary_points.unsqueeze(0), dim=2)
print(len(non_boundary_points))
#non_boundary_mask = ~((non_boundary_points.unsqueeze(1) == boundary_points.unsqueeze(0)).all(dim=2).any(dim=1))
boundary_tensor = boundary_points.unsqueeze(1).repeat(1, non_boundary_points.size(0), 1)
non_boundary_mask = ~((non_boundary_points.unsqueeze(0) == boundary_tensor).all(dim=2).any(dim=0))

non_boundary_points = non_boundary_points[non_boundary_mask]
non_boundary_labels = non_boundary_labels[non_boundary_mask]

# Combine boundary and non-boundary points and labels
all_points = torch.cat([boundary_points, non_boundary_points], dim=0)
all_labels = torch.cat([boundary_labels, non_boundary_labels], dim=0)

# Combine data and labels into a TensorDataset
dataset = TensorDataset(all_points, all_labels)

# Shuffle the dataset
shuffled_indices = torch.randperm(len(dataset))
dataset = dataset[shuffled_indices]
train_ratio = 0.8
# Split the data into training and validation sets
train_size = int(len(dataset) * train_ratio)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])
batch_size= 32
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size, shuffle=False)

5000000
